In [1]:
!pip install jupyter-dash
!pip install dash-bio==1.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import scipy
import h5py
import pandas as pd
import dash_bio

## Load data

In [3]:
with h5py.File('genes_data.h5', 'r') as fd:
    fd_hg = fd['FD_HG'][:]
    fd_lg = fd['FD_LG'][:]
    fd_n = fd['FD_N'][:]
    fd_label = [x.decode() for x in fd['FD_Labels']]


print(f'FD_HG: {fd_hg.shape}')
print(f'FD_LG: {fd_lg.shape}')
print(f'FD_N: {fd_n.shape}')



FD_HG: (12, 1920)
FD_LG: (12, 1920)
FD_N: (17, 1920)


Define functions



In [4]:
def preprocess_fd(fd):
    if fd.shape[0] > 1:
        fd = np.mean(fd, axis=0)
    return fd


def fold_change(x, y, use_log=False):
    if use_log:
        x = np.log2(x)
        y = np.log2(y)
    return x - y  # difference of logarithms


def filter_genes(x, gtype):
    if gtype == 'up':
        return 

# Experiments

We choose one of the three experimental settings:

- HG vs N
- LG vs N
- (HG $\cup$ LG) vs N

## HG vs N 

In [ ]:
current_fd = fd_hg
exp_fd = preprocess_fd(fd_hg)
exp_fd_n = preprocess_fd(fd_n)

## LG vs L

In [ ]:
current_fd = fd_lg
exp_fd = preprocess_fd(fd_lg)
exp_fd_n = preprocess_fd(fd_n)

## (HG $\cup$ LG) vs N

In [5]:
current_fd = np.concatenate([fd_lg, fd_hg], axis=0)
exp_fd = preprocess_fd(current_fd)
exp_fd_n = preprocess_fd(fd_n)

Computation to obtain Volcano Plot parameters:

* Log Fold Change
* p-value by Welch's t-test

In [6]:
# Compute fold change
log_fold_change = fold_change(exp_fd, exp_fd_n) 

# Calculate the Welch's t-test of two independent samples of scores
fd_n_pv = scipy.stats.ttest_ind(current_fd, fd_n, equal_var=False).pvalue

Set Fold-Change and volcano plot parameters

* **lfc_thr:** log fold change threshold
* **pv_thr:** p-value threshold

In [7]:
lfc = 'expsFC'
pv = 'pv'
fc_value = 2
lfc_thr = (-np.log2(fc_value), np.log2(fc_value))
pv_thr = 0.001

Produce volcano plot

In [31]:
df = pd.DataFrame({
    'expsFC': log_fold_change,
    'pv': fd_n_pv,
    'genes_idx': list(range(log_fold_change.shape[0])),
    'genes_label': fd_label
})


dash_bio.VolcanoPlot(dataframe=df, effect_size='expsFC', p='pv', snp='genes_idx',
                     gene='genes_label', effect_size_line=lfc_thr, 
                     genomewideline_value=-np.log10(pv_thr), xlabel='log2(FC)')

Filter fold change and return upregulated and downregulated

In [12]:
# Get up/down regulated
upreg = df.loc[(df[lfc] >= -lfc_thr[0]) & (df[pv] < pv_thr)]
downreg = df.loc[(df[lfc] <= -lfc_thr[1]) & (df[pv] < pv_thr)]

print("Total up/down regulated genes,", upreg.shape[0] + downreg.shape[0])

Total up/down regulated genes, 131


Filter out duplicated genes retaining most significant fold change based on p-value score


In [16]:
selected_genes = pd.concat([upreg, downreg]).sort_values(by='pv')

group_sg = selected_genes.groupby('genes_label')

new_genes = pd.DataFrame()

for i, gene in enumerate(group_sg['genes_label']):
    genes = selected_genes.loc[selected_genes['genes_label'] == gene[0]]
    genes = genes.drop_duplicates(subset='genes_label', keep='last')
    new_genes = pd.concat([new_genes, genes], )

new_genes = new_genes.sort_values(by='pv')

print("Filtered genes:", new_genes.shape)
new_genes.head()

Filtered genes: (70, 4)


,expsFC,pv,genes_idx,genes_label
242,2.734768,9.549333e-13,242,uc.8+
572,1.312917,2.148362e-11,572,uc.369+
86,1.318176,2.625789e-11,86,uc.346+
68,1.066556,5.321091e-09,68,uc.466+A
420,-1.908603,1.036398e-08,420,uc.354+A


Save output samples

In [51]:
fd_n_filtered = fd_n[:, new_genes['genes_idx']] 
fd_lg_filtered = fd_lg[:, new_genes['genes_idx']] 
fd_hg_filtered = fd_hg[:, new_genes['genes_idx']] 

x = pd.DataFrame(np.vstack([fd_n_filtered, fd_lg_filtered, fd_hg_filtered]))

y = pd.DataFrame([i for i in range(x.shape[0])])
labels = new_genes.sort_values(by=['genes_idx'])['genes_label']

x.to_csv('X_0001_2.csv')
y.to_csv('Y.csv')
labels.to_csv('Labels_0001_2.csv')